In [2]:
import time
import neptune # type: ignore
import datetime
import torch
import torch.optim as optim
import torch.nn as nn

from torch.optim import SGD, Adam # type: ignore
from torch.optim.lr_scheduler import LambdaLR # type: ignore
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import VOCSegmentation
from torchmetrics import Accuracy # type: ignore

from configs.utils import *
from datasets.utils import *
from datasets.utils import FlairDs
from easydict import EasyDict # type: ignore
from argparse import ArgumentParser, Namespace
from model.segmenter_adapt import SegmenterAdapt
from dl_toolbox.callbacks import EarlyStopping # type: ignore
from model.segmenter_adapt import *
from configs.utils import *
from datasets.utils import *
from easydict import EasyDict

ImportError: cannot import name 'File' from 'neptune.types' (c:\Users\marie\anaconda3\envs\css\lib\site-packages\neptune\types\__init__.py)

In [ ]:
args = Namespace(
    initial_lr=0.01,
    final_lr=0.005,
    lr_milestones=(20, 80),
    epoch_len=100,
    sup_batch_size=4,
    crop_size=256,
    workers=6,
    img_aug='d4_rot90_rot270_rot180_d1flip',
    max_epochs=200,
    sequence_path="",
    train_split_coef=0.85,
    strategy='continual_{}',
    commit=None,
    train_type="adaptmlp",
    replay=False,  # Par défaut, `store_true` est `False`
    config_file="/d/maboum/JSTARS/segmentation/configs/config.yml",
    ffn_adapt=True,  # Par défaut, `store_true` est `True`
    ffn_num=64,
    vpt=False,  # Par défaut, `store_true` est `False`
    vpt_num=1,
    fulltune=False  # Par défaut, `store_true` est `False`
)

tuning_config = EasyDict(
        # AdaptFormer
        ffn_adapt=args.ffn_adapt,
        ffn_option="parallel",
        ffn_adapter_layernorm_option="none",
        ffn_adapter_init_option="lora",
        ffn_adapter_scalar="0.1",
        ffn_num=args.ffn_num,
        d_model=384,
        # VPT related
        vpt_on=args.vpt,
        vpt_num=args.vpt_num,
        nb_task = 3
    )
config_file = "C:/Users/marie/css-peft/configs/config.yml"
config = load_config_yaml(file_path = config_file)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = config["dataset"]
data_config = dataset["flair1"]
seed = config["seed"]
directory_path = data_config["data_path"]
metadata = data_config["metadata"]
data_sequence = data_config["task_name"]
epochs = data_config['epochs']
eval_freq = data_config['eval_freq']
im_size = data_config["im_size"]
lr = data_config['learning_rate']
win_size = data_config["window_size"]
win_stride = data_config["window_stride"]
n_channels = data_config['n_channels']
n_class = 13
class_names = data_config["classnames_binary"]
eval_freq = data_config["eval_freq"]

selected_model = '_'.join([config["model_name"], "224"])
model = config["model"]
model_config = model[selected_model]
im_size = model_config["image_size"]
patch_size = model_config["patch_size"]
d_model = model_config["d_model"]
n_heads = model_config["n_heads"]
n_layers = model_config["n_layers"]
d_encoder = model_config["d_model"]

train_type = args.train_type
lora_params = config["lora_parameters"]
lora_rank = lora_params["rank"]
lora_alpha = lora_params["rank"]

binary = data_config["binary"]

In [ ]:
seed = config["seed"]
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.autograd.set_detect_anomaly(True) 
random.seed(seed)

In [ ]:
run = neptune.init_run(
        project="continual-semantic-segmentation/peft-methods",
        api_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwN2IzOGYxMC0xYTg5LTQxMGEtYjE3Yy1iNDVkZDM1MmEzYzIifQ==",
        name="AdaptFormerSeg",
        description="First run for Adapters project",
        tags=["adaptmlp", "test", "segmenter", "vit-large"],
        mode= "debug")

In [ ]:
# Learning rate
def lambda_lr(epoch):
    m = epoch / args.max_epochs
    if m < args.lr_milestones[0]:
        return 1
    elif m < args.lr_milestones[1]:
        return 1 + ((m - args.lr_milestones[0]) / (
                    args.lr_milestones[1] - args.lr_milestones[0])) * (
                            args.final_lr / args.initial_lr - 1)
    else:
        return args.final_lr / args.initial_lr

In [ ]:
# Callbacks
segmentation_model = SegmenterAdapt(im_size, n_layers, d_model, d_encoder, 4 * d_model, n_heads, n_class,
                                                patch_size, selected_model, tuning_config=tuning_config,
                                                model_name=config["model_name"], id = 0).to(device)
segmentation_model.load_pretrained_weights()
segmentation_model_path = os.path.join("C:/Users/marie/css-peft/checkpoints",args.sequence_path.format(seed),
                                 '{}_{}'.format(args.strategy.format(train_type),seed))
early_stopping = EarlyStopping(patience=20, verbose=True, 
                                delta=0.001,path=segmentation_model_path)
optimizer = SGD(segmentation_model.parameters(),
                lr=args.initial_lr,
                momentum=0.9)
scheduler = LambdaLR(optimizer,lr_lambda= lambda_lr, verbose = True)
loss_fn = torch.nn.CrossEntropyLoss().cuda()
accuracy = Accuracy(task='multiclass',num_classes=n_class).cuda()

In [ ]:
train_imgs, test_imgs = [],[]
for step,domain in enumerate(data_sequence[:2]):

    img = glob.glob(os.path.join(directory_path, '{}/Z*_*/img/IMG_*.tif'.format(domain)))
    random.shuffle(img)
    train_imgs += img[:int(len(img)*args.train_split_coef)]
    test_imgs += img[int(len(img)*args.train_split_coef):]
    random.shuffle(train_imgs)
    # Train&Validation Data
    domain_img_train = train_imgs[:int(len(train_imgs)*args.train_split_coef)]
    domain_img_val = train_imgs[int(len(train_imgs)*args.train_split_coef):]
    train_loader = create_train_dataloader(domain_img_train, args, data_config, binary= binary)
    val_loader = create_val_dataloader(domain_img_val, args, data_config, binary= binary)

    for param in segmentation_model.encoder.parameters():
        param.requires_grad = False
    # Unfreeze the adapt_mlp layers
    for name, param in segmentation_model.encoder.named_parameters():
        if 'adaptmlp' in name:
            param.requires_grad = True

    for epoch in range(1,args.max_epochs):
        time_ep = time.time()
        segmentation_model, train_acc, train_loss = train_function(segmentation_model,train_loader, 
                                            device,optimizer, loss_fn,
                                            accuracy, epoch, data_config, run)
        print(train_acc, train_loss)
        scheduler.step()
        segmentation_model, val_loss, val_acc = validation_function(segmentation_model,val_loader, 
                                                        device,loss_fn,
                                                        accuracy, epoch, data_config, run)
        print(val_acc, val_loss)
        early_stopping(val_loss,segmentation_model)
        if early_stopping.early_stop:
            break
        time_ep = time.time() - time_ep
    segmentation_model.increment()
